# Libbitcoin BX: Mnemonics & HD Wallets
In this BX tutorial, we create mnemonic backup phrases, from which HD private and public keys can be derived for use. The mnemonic phrase can later be used to recover hd keys used in a previous wallet with the same seed.

## 1) Seeding an HD wallet.
<img src="images/hd_mnemonic_to_master.jpg" alt="drawing" style="" width="700px"/>

### 1.1) Deriving the mnemonic phrase (BIP39)

In [ ]:
bx seed --bit_length 128


In [ ]:
# entropy for mnemonic encoding must be divisible by 32 bits.
bx mnemonic-new --language ja 04967dfa6011c1eefc296a865702641b


In [ ]:
# mnemonic-to-seed always derives a 512-bit long seed.
bx mnemonic-to-seed --language ja むける りりく あんぜん ひろい よかぜ いっぽう でぬかえ むいか うんてん げいのうじん ひほう きぞく


## 1.2) Deriving the master HD keys (BIP32)

In [ ]:
# accepts seed 128bit and longer.
bx hd-new 82a09fb73fd4256e007c0037cc9ead05cacef87daf8d20dff4de8d8d03b75c0d2e82bc3dcd5630c6276e2509e7a84bac86021136685645efd09975c1c48e5cce


In [ ]:
bx hd-to-public tprv8ZgxMBicQKsPfEZdr7dK8QypakZdLYipbGzhbMPjiHrWWRhCpBi1k42MUs4bZ29zj3NyHS34pp58NMUgGBLjMxvKHpsx8ZdigjDhZxzXzrB


## 2) Deriving HD children
<img src="images/hd_children_derivation.jpg" alt="drawing" style="" width="700px"/>

### 2.1) Deriving unhardened children hd keys.

**`m` &#8658; `m/0/1/2`** `(private-key child derivation)`

In [ ]:
bx hd-private --index 0 tprv8ZgxMBicQKsPfEZdr7dK8QypakZdLYipbGzhbMPjiHrWWRhCpBi1k42MUs4bZ29zj3NyHS34pp58NMUgGBLjMxvKHpsx8ZdigjDhZxzXzrB \
| bx hd-private --index 1 \
| bx hd-private --index 2 


**`M` &#8658; `M/0/1/2`** `(public-key child derivation)`

In [ ]:
bx hd-public --index 0  tpubD6NzVbkrYhZ4YhbRjmHuXpdw9n5ZVsujAabUssS38ZeuLuwySaXbvYeDezqYHcFc5JauHX2WMEK4AFwN1fEbUmcw4jmPpQnRAhd8nL1gpFM \
| bx hd-public --index 1 \
| bx hd-public --index 2 


**`m/0/1/2` &#8658; `M/0/1/2`** 

In [ ]:
#(compare key with preceding example)
bx hd-to-public tprv8fgiWAFTz5LuSKAcgnrDF1HUjvgcfaA2y3quyPxxubvzDhvV7GB8TNRGP6RRSivyvsRsD9Tyey99QjrYBPLBvDAqqZb5YXFaWuiACgEMjFp


**Payment addresses from hd-keys**
* `hd-to-ec` derives public key from extended public key.
* `ec-to-address` returns payment addresss from public key.
    * `--version` : `0x6f` = `111` in decimal

In [ ]:
bx hd-to-ec tpubDCNkeaHi8T2aKnCQaSWoeQwbJxCYpuLwYMShFv1GKsjP4CBFjezids38ZDnL9owHpzNpUnAT5nF5h9PKbFmLBTsjwBDpUS2ZtCXzkucFTLQ \
| bx ec-to-address --version 111


### 2.2) Deriving hardened children hd keys.

**`m` &#8658; `m/44'`** 

In [ ]:
bx hd-private --hard --index 44 tprv8ZgxMBicQKsPfEZdr7dK8QypakZdLYipbGzhbMPjiHrWWRhCpBi1k42MUs4bZ29zj3NyHS34pp58NMUgGBLjMxvKHpsx8ZdigjDhZxzXzrB


**`m` &#8658; `M` &#8658; `M/44'`** 

In [ ]:
bx hd-public --hard --index 44 tprv8ZgxMBicQKsPfEZdr7dK8QypakZdLYipbGzhbMPjiHrWWRhCpBi1k42MUs4bZ29zj3NyHS34pp58NMUgGBLjMxvKHpsx8ZdigjDhZxzXzrB


**`m/44'` &#8658; `M/44'/1'`** 

In [ ]:
bx hd-private --hard --index 1 tprv8d8Rvn9LCWesLLks73oz5bhZ2Wp3j8paJL9jYEDRpUT4LmxAtkXyuJF3VpLNDm5cHU8jfeUusbvoYXDidrffFbUmkxXLJJsTybCJBzJKd6R


**Try: `M/44'` &#8658; `M/44'/1'`**

In [ ]:
bx hd-public --hard --index 1 tpubD9pU5CBaLtLYDonezhUaV1MfbYKytU1UsdkWpkFjEkFTBGCwX9Ma5nrufxte6NR6K78XpPWoTXSzhEFrRnGDyM8FMFVbzc8jqcMAim2aBZd 


## 3) Recovering a HD wallet from mnemonic phrase
<img src="images/hd_wallet_recovery.jpg" alt="drawing" style="" width="700px"/>

**In order to recover previously used hd-derivation paths of an hd-wallet:**
* Derive hd parent keys of receiving & change addresses from mnemonic (& passphrase).
    * Parent of receiving addresses
        * `m/44'/0'/0'/0` (mainnet)
        * `m/44'/1'/0'/0` (testnet)
    * Parent of change addresses
        * `m/44'/0'/0'/1` (mainnet)
        * `m/44'/1'/0'/1` (testnet)        
* Increment hd-key index of children until unused addresses are found.
    * Query Libbitcoin Server to check history of a payment address
        * `bx fetch-history` `[payment address]`

In [ ]:
mnemonic='むける りりく あんぜん ひろい よかぜ いっぽう でぬかえ むいか うんてん げいのうじん ひほう きぞく'
m_testnet=$(bx mnemonic-to-seed --language ja $mnemonic | bx hd-new)
m_44h_1h_0h_0=$(bx hd-private --index 44 --hard $m_testnet \
| bx hd-private --index 1 --hard \
| bx hd-private --index 0 --hard \
| bx hd-private --index 0)


gap_count=0
current_index=0

while (( $gap_count < 20 ))
do

    current_address=$(bx hd-private --index $current_index $m_44h_1h_0h_0 \
    | bx hd-to-public \
    | bx hd-to-ec \
    | bx ec-to-address --version 111)
     
    query_address_history_reply=$(bx fetch-history $current_address)

    if  [ "$query_address_history_reply" = 'transfers ""' ]
    then
        echo address at index $current_index is unused.
        let gap_count++
    
    # reset gap counter if gap interrupted before it reaches 20.
    elif (( $gap_count > 0 ))
    then 
        gap_count=0
    fi
    
    let current_index++
    
done